# Samhliðamálheild fyrir ensku og íslensku - ParIce
Hér er unnið með ParIce málheildina. Gangasafnið lesið, prófunargögn síuð út og sett í staðlað form. 

Gert er ráð fyrir að eftirfarandi gögn séu til staðar
- ParIce eins og það er sótt á málföng, `parice_dir`
- Prófunargögn eins og fengin frá Steinþóri, `test_dir`

Eftir að hafa keyrt reikniritið verður `target_dir` eftirfarandi
- `target_dir/x.(is|en)`, þar sem `x` er undirmálheild úr ParIce.
- `target_dir/train.(is|en)` allar málheildir úr ParIce sameinaðar og prófunar- og fínpússunargögn fjarlægð.
- `target_dir/dev.(is|en)` 2000 setningapör valin af handahófi úr sameinuðu ParIce með prófunargögn fjarlægð.
- `target_dir/test.(is|en)` Prófunargögn eins og fengin frá Steinþóri. 

In [ ]:
# The location of Parice to read
parice_dir = '/work/haukurpj/data/raw/Parice1.0'
# The location of the Parice test data (zipped, provided separatly).
test_set_zip = '/work/haukurpj/data/raw/prófunarsett-20191205T064220Z-001.zip'

# The location of where to write the results
target_dir = '/work/haukurpj/data/formatted/Parice1.0'

from glob import glob
from pprint import pprint

In [ ]:
import pathlib

parice_raw_dir = pathlib.Path(parice_dir)
parice_formatted_dir = pathlib.Path(target_dir)
parice_test_zip = pathlib.Path(test_set_zip)

In [ ]:
assert parice_raw_dir.exists()
assert parice_test_zip.exists()
if not parice_formatted_dir.exists():
    parice_formatted_dir.mkdir()

## Sækja opensubtitles
Opensubtitles er ekki deilt með ParIce, út af leyfismálum, svo við þurfum að sækja þau gögn og keyra scriptu til þess að para setningar.

In [11]:
!wget -P {parice_raw_dir}/opensubtitles/en.zip http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/raw/en.zip
!wget -P {parice_raw_dir}/opensubtitles/is.zip http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/raw/is.zip

--2020-01-17 09:58:51--  http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/raw/en.zip
Resolving opus.nlpl.eu (opus.nlpl.eu)... 193.166.25.9
Connecting to opus.nlpl.eu (opus.nlpl.eu)|193.166.25.9|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/raw/en.zip [following]
--2020-01-17 09:58:52--  https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/raw/en.zip
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.19, 86.50.254.18
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13736002649 (13G) [application/zip]
Saving to: ‘/work/haukurpj/data/formatted/Parice1.0/download.php?f=OpenSubtitles%2Fv2018%2Fraw%2Fen.zip’

download.php?f=Open 100%[===================>]  12.79G  7.51MB/s    in 26m 1s  

2020-01-17 10:24:53 (8.39 MB/s) - ‘/work/haukurpj/data/formatted/Parice1.0/download.php?f=OpenSubtitles%2Fv20

In [30]:
!mkdir -p {parice_raw_dir}/opensubtitles/en
!mkdir -p {parice_raw_dir}/opensubtitles/is

In [ ]:
!unzip {parice_raw_dir}/en.zip -d {parice_raw_dir}/en
!unzip {parice_raw_dir}/is.zip -d {parice_raw_dir}/is

Núna keyrum við scriptuna sem fylgir með ParIce til þess að para saman setningar.

### Virkar ekki!
The script gives an error which seems to be related to the code... Instead we cheat and copy opensubtitles from what Steinþór sent us.

In [35]:
%cd {parice_raw_dir}/opensubtitles
!python read_opus.py --input {parice_raw_dir}/opensubtitles --output {parice_raw_dir}/tmx

/work/haukurpj/data/raw/Parice1.0/opensubtitles
21079 1931
1:0 1 2 3 4 5 6 |8 9 10 11 12
 0:2 3 4 5 6 |8 9 10 11 12

[['1', 'Allir, sem sverði bregða, munu fyrir sverði falla.'], ['2', 'Matteus 26-52'], ['3', 'Hvernig er þetta?'], ['4', 'Þú mátt eiga það, Rico.'], ['5', 'Kollurinn er enn í lagi.'], ['6', '- Hvað má bjóða ykkur?'], ['7', '- Spagetti og kaffi handa tveimur.'], ['8', 'Allt í lagi, herra.'], ['9', '"Undirheimar votta demanta Pete Montana virðingu sína."'], ['10', 'Hvað kemur það málinu við?'], ['11', 'Helling.'], ['12', 'Demanta Pete Montana.'], ['13', 'Hann þarf ekki að sóa tímanum á bensínstöðvum.'], ['14', 'Hann er þekktur.'], ['15', 'Er í borginni og lifir hátt.'], ['16', 'Og sjá okkur.'], ['17', 'Algjörir ræflar, einskis virði.'], ['18', 'Viltu þetta, Rico?'], ['19', 'Viltu svona veislu fyrir þig?'], ['20', '"Caesar Enrico Bandello.'], ['21', 'Hylltur af vinum sínum."'], ['22', 'Ég gæti gert allt sem hann gerir og meira til.'], ['23', 'En ég fékk aldrei tækifæri.'], [

In [47]:
!ls {parice_raw_dir}/tmx

baekur.tmx   ema.tmx	    hagstofan.tmx      tatoeba.tmx
biblian.tmx  eso.tmx	    kde4.tmx	       ubuntu.tmx
ees.tmx      fornritin.tmx  opensubtitles.tmx


## Lesa .tmx skrár
Fyrst þarf þó aðeins að lagfæra skrárnar svo þær séu allar með sama tungumála merki.

In [44]:
!grep -m 2 "lang=\"is\"" {parice_raw_dir}/tmx/*.tmx
!grep -m 2 "lang=\"en\"" {parice_raw_dir}/tmx/*.tmx

/work/haukurpj/data/raw/Parice1.0/tmx/ema.tmx:      <tuv xml:lang="is"><seg>" samkvæmt birtum niðurstöðum</seg></tuv>
/work/haukurpj/data/raw/Parice1.0/tmx/ema.tmx:      <tuv xml:lang="is"><seg>" HbA1c (%) í 24. viku.</seg></tuv>
/work/haukurpj/data/raw/Parice1.0/tmx/hagstofan.tmx:      <tuv xml:lang="is"><seg>Nánar um útgáfu</seg></tuv>
/work/haukurpj/data/raw/Parice1.0/tmx/hagstofan.tmx:      <tuv xml:lang="is"><seg>Skráðir nemendur í framhalds- og háskólum haustið 2010</seg></tuv>
/work/haukurpj/data/raw/Parice1.0/tmx/ema.tmx:  <header adminlang="EN-US" creationdate="20170426T083707Z" creationtool="Moses-to-TMX-converer" creationtoolversion="1.0" datatype="plaintext" o-tmf="Moses plain text files" segtype="sentence" srclang="en">
/work/haukurpj/data/raw/Parice1.0/tmx/ema.tmx:      <tuv xml:lang="en"><seg>" based on published data</seg></tuv>
/work/haukurpj/data/raw/Parice1.0/tmx/hagstofan.tmx:  <header adminlang="en" creationdate="20170319T213035Z" creationtool="mALIGNa" creationtoo

In [45]:
!sed -i 's/xml:lang="is"/xml:lang="IS-IS"/g' {parice_raw_dir}/tmx/hagstofan.tmx
!sed -i 's/xml:lang="en"/xml:lang="EN-GB"/g' {parice_raw_dir}/tmx/hagstofan.tmx
!sed -i 's/xml:lang="is"/xml:lang="IS-IS"/g' {parice_raw_dir}/tmx/ema.tmx
!sed -i 's/xml:lang="en"/xml:lang="EN-GB"/g' {parice_raw_dir}/tmx/ema.tmx

In [46]:
!grep -m 2 "lang=\"is\"" {parice_raw_dir}/tmx/*.tmx
!grep -m 2 "lang=\"en\"" {parice_raw_dir}/tmx/*.tmx

/work/haukurpj/data/raw/Parice1.0/tmx/ema.tmx:  <header adminlang="EN-US" creationdate="20170426T083707Z" creationtool="Moses-to-TMX-converer" creationtoolversion="1.0" datatype="plaintext" o-tmf="Moses plain text files" segtype="sentence" srclang="en">
/work/haukurpj/data/raw/Parice1.0/tmx/hagstofan.tmx:  <header adminlang="en" creationdate="20170319T213035Z" creationtool="mALIGNa" creationtoolversion="2" datatype="plaintext" o-tmf="al" segtype="block" srclang="en">


In [53]:
tmx_files = glob(f'{parice_raw_dir}/tmx/*.tmx')
tmx_files = [pathlib.Path(tmx_file) for tmx_file in tmx_files]
pprint(tmx_files)

[PosixPath('/work/haukurpj/data/raw/Parice1.0/tmx/ema.tmx'),
 PosixPath('/work/haukurpj/data/raw/Parice1.0/tmx/biblian.tmx'),
 PosixPath('/work/haukurpj/data/raw/Parice1.0/tmx/fornritin.tmx'),
 PosixPath('/work/haukurpj/data/raw/Parice1.0/tmx/opensubtitles.tmx'),
 PosixPath('/work/haukurpj/data/raw/Parice1.0/tmx/ees.tmx'),
 PosixPath('/work/haukurpj/data/raw/Parice1.0/tmx/tatoeba.tmx'),
 PosixPath('/work/haukurpj/data/raw/Parice1.0/tmx/baekur.tmx'),
 PosixPath('/work/haukurpj/data/raw/Parice1.0/tmx/kde4.tmx'),
 PosixPath('/work/haukurpj/data/raw/Parice1.0/tmx/hagstofan.tmx'),
 PosixPath('/work/haukurpj/data/raw/Parice1.0/tmx/eso.tmx'),
 PosixPath('/work/haukurpj/data/raw/Parice1.0/tmx/ubuntu.tmx')]


In [56]:
import frontend.bulk as b

parice_tmx = b.tmx_split(tmx_files, 'EN-GB', 'IS-IS')
en_parice, is_parice = zip(*parice_tmx)

In [58]:
for corpus in en_parice + is_parice:
    !mv {corpus} {parice_formatted_dir}

In [4]:
parice_en_list = glob(f'{parice_formatted_dir}/*.en')
parice_is_list = glob(f'{parice_formatted_dir}/*.is')
parice_en_list.sort()
parice_is_list.sort()
pprint(parice_en_list)
pprint(parice_is_list)

['/work/haukurpj/data/formatted/Parice1.0/baekur.en',
 '/work/haukurpj/data/formatted/Parice1.0/biblian.en',
 '/work/haukurpj/data/formatted/Parice1.0/ees.en',
 '/work/haukurpj/data/formatted/Parice1.0/ema.en',
 '/work/haukurpj/data/formatted/Parice1.0/eso.en',
 '/work/haukurpj/data/formatted/Parice1.0/fornritin.en',
 '/work/haukurpj/data/formatted/Parice1.0/hagstofan.en',
 '/work/haukurpj/data/formatted/Parice1.0/kde4.en',
 '/work/haukurpj/data/formatted/Parice1.0/opensubtitles.en',
 '/work/haukurpj/data/formatted/Parice1.0/tatoeba.en',
 '/work/haukurpj/data/formatted/Parice1.0/ubuntu.en']
['/work/haukurpj/data/formatted/Parice1.0/baekur.is',
 '/work/haukurpj/data/formatted/Parice1.0/biblian.is',
 '/work/haukurpj/data/formatted/Parice1.0/ees.is',
 '/work/haukurpj/data/formatted/Parice1.0/ema.is',
 '/work/haukurpj/data/formatted/Parice1.0/eso.is',
 '/work/haukurpj/data/formatted/Parice1.0/fornritin.is',
 '/work/haukurpj/data/formatted/Parice1.0/hagstofan.is',
 '/work/haukurpj/data/form

## Sía út prófunargögn
Núna síum við út prófunargögnin úr ParIce. Byrjum á því að afpakka.

In [63]:
!unzip {parice_test_zip} -d {parice_formatted_dir}

Archive:  /work/haukurpj/data/raw/prófunarsett-20191205T064220Z-001.zip
  inflating: /work/haukurpj/data/formatted/Parice1.0/pr?funarsett/LESTU-MIG.docx  
  inflating: /work/haukurpj/data/formatted/Parice1.0/pr?funarsett/ees.csv  
  inflating: /work/haukurpj/data/formatted/Parice1.0/pr?funarsett/opensubtitles.csv  
  inflating: /work/haukurpj/data/formatted/Parice1.0/pr?funarsett/ema.csv  


In [67]:
!ls {parice_formatted_dir}/prófunarsett

LESTU-MIG.docx	ees.csv  ema.csv  opensubtitles.csv


In [5]:
def read_csv(p_in):
    en_text = []
    is_text = []
    with p_in.open() as f_in:
        for index, line in enumerate(f_in):
            if '### SKJAL ID' in line:
                continue
            if line == '\n':
                continue
            id, en_sent, is_sent = line.split('\t')
            en_text.append(en_sent+'\n')
            is_text.append(is_sent)
    return en_text, is_text

In [6]:
ees_en, ees_is = read_csv(parice_formatted_dir.joinpath('prófunarsett').joinpath('ees.csv'))
ema_en, ema_is = read_csv(parice_formatted_dir.joinpath('prófunarsett').joinpath('ema.csv'))
opensubtitles_en, opensubtitles_is = read_csv(parice_formatted_dir.joinpath('prófunarsett').joinpath('opensubtitles.csv'))
parice_test_is = ees_is + ema_is + opensubtitles_is
parice_test_en = ees_en + ema_en + opensubtitles_en

In [7]:
parice_is = []
for corpus in parice_is_list:
    with open(corpus) as f_in:
        parice_is += f_in.readlines()

parice_en = []
for corpus in parice_en_list:
    with open(corpus) as f_in:
        parice_en += f_in.readlines()

In [8]:
pprint(len(parice_is))
pprint(len(parice_en))
for i in range(2):
    print(parice_is[i], parice_en[i])

3545822
3545822
FYRRI ÞÁTTUR
 BOOK I

I. Veizlan í nýju höllinni.
 CHAPTER I A FETE AT THE NEW PALACE



In [9]:
def get_indices(corpus, sentences):
    indices = []
    for index, line in enumerate(corpus):
        if line in sentences:
            indices.append(index)
    return indices

In [10]:
indicies_test_en = get_indices(parice_en, frozenset(parice_test_en))
indicies_test_is = get_indices(parice_is, frozenset(parice_test_is))
print(len(indicies_test_en), len(indicies_test_is))

178207 193264


Núna þurfum við að fjarlægja setningar úr ParIce sem eru í prófunargögnunum.

In [11]:
indicies_test_intersection = set(indicies_test_is).intersection(set(indicies_test_en))
indicies_test_union = set(indicies_test_is).union(set(indicies_test_en))
print(len(indicies_test_intersection))
print(len(indicies_test_union))
print(len(parice_test_en), len(parice_test_is))

139384
232087
7645 7645


Við notum sammengið, til þess að losa okkur við allan efa um að þýðingarvélin hafi séð þessar setningar áður og einnig til þess að vera með sambærilega vinnslu og Miðeind.

In [12]:
def drop_indices(corpus, indices):
    filtered_corpus = []
    indices = list(indices)
    indices.sort()
    current_index = 0
    for corpus_index, line in enumerate(corpus):
        if current_index == len(indices):
            filtered_corpus.append(line)
        elif indices[current_index] == corpus_index:
            current_index += 1
        else:
            filtered_corpus.append(line)
    return filtered_corpus

In [13]:
drop = indicies_test_union
parice_no_test_en = drop_indices(parice_en, drop)
parice_no_test_is = drop_indices(parice_is, drop)

## Skipta í dev/train
Við skiptum restini af gögnunum í tvennt, dev/train. Við veljum 2000 setningapör í dev og fjarlægjum þær úr restinni. Sá afgangur verður train. Það er hugsanlegt að við veljum tvítekningar í dev en við verðum bara að lifa með það.

In [14]:
seed = 10

import random
random.seed(seed)
parice_dev_en = list(random.sample(parice_no_test_en, 2000))
indieces_dev_en = get_indices(parice_no_test_en, frozenset(parice_dev_en))
random.seed(seed)
parice_dev_is = list(random.sample(parice_no_test_is, 2000))
indieces_dev_is = get_indices(parice_no_test_is, frozenset(parice_dev_is))

In [15]:
print(len(indieces_dev_en), len(indieces_dev_is))

74037 73147


Hægt er að keyra kóðann (illa skrifaður) fyrir neðan til þess að athuga hvort niðurstöðurnar eru réttar. Þær virðast vera það.

In [ ]:
for line in parice_dev_en:
    print(line)
    for index, p_line in enumerate(parice_no_test_en):
        if p_line == line:
            print(index, p_line)
            if index not in indieces_dev_en:
                raise RuntimeError("Error! Not found")

In [16]:
indicies_dev_intersection = set(indieces_dev_en).intersection(set(indieces_dev_is))
indicies_dev_union = set(indieces_dev_en).union(set(indieces_dev_is))
print(len(indicies_dev_intersection))
print(len(indicies_dev_union))
print(len(parice_dev_en), len(parice_dev_is))

63725
83459
2000 2000


In [17]:
for i in range(2):
    print(parice_dev_en[i], parice_dev_is[i])

Get in the right lane.
 Farðu á hægri akrein.

• Where the Commission ascertains that a notified body does not meet or no longer meets the requirements for its notification, it shall adopt an implementing act requesting the notifying Member State to take the necessary corrective measures, including withdrawal of notification if necessary.
 • Ef framkvæmdastjórnin kemst að raun um að tilkynnt stofa uppfylli ekki, eða uppfylli ekki lengur, kröfurnar viðvíkjandi tilkynningu hennar, skal hún samþykkja framkvæmdargerð þar sem óskað er eftir því að tilkynningaraðildarríkið geri nauðsynlegar ráðstafanir til úrbóta, þ.m.t. að afturkalla tilkynningu, ef nauðsyn krefur.



In [18]:
drop = indicies_dev_union
parice_train_en = drop_indices(parice_no_test_en, drop)
parice_train_is = drop_indices(parice_no_test_is, drop)

Athugum hvort línufjöldinn gangi ekki upp.

In [20]:
print(len(parice_train_en), len(parice_train_is))
print(len(parice_en) - (len(parice_train_en) + len(indicies_dev_union) + len(indicies_test_union)))

3230276 3230276
0


## Skrifa gögn og hreinsa til
Núna skrifum við gögnin niður í skrár og fjarlægjum tímabundnar skrár.

In [21]:
!ls {target_dir}

baekur.en   ees.en  eso.en	  hagstofan.en	opensubtitles.en  tatoeba.is
baekur.is   ees.is  eso.is	  hagstofan.is	opensubtitles.is  ubuntu.en
biblian.en  ema.en  fornritin.en  kde4.en	prófunarsett	  ubuntu.is
biblian.is  ema.is  fornritin.is  kde4.is	tatoeba.en


In [22]:
def write_to_file(corpus, path):
    with path.open('w+') as f_in:
        for line in corpus:
            f_in.write(line)
    !wc -l {path}

In [24]:
write_to_file(parice_train_en, parice_formatted_dir.joinpath('train.en'))
write_to_file(parice_train_is, parice_formatted_dir.joinpath('train.is'))
write_to_file(parice_dev_en, parice_formatted_dir.joinpath('dev.en'))
write_to_file(parice_dev_is, parice_formatted_dir.joinpath('dev.is'))
write_to_file(parice_test_en, parice_formatted_dir.joinpath('test.en'))
write_to_file(parice_test_is, parice_formatted_dir.joinpath('test.is'))

3230276 /work/haukurpj/data/formatted/Parice1.0/train.en
3230276 /work/haukurpj/data/formatted/Parice1.0/train.is
2000 /work/haukurpj/data/formatted/Parice1.0/dev.en
2000 /work/haukurpj/data/formatted/Parice1.0/dev.is
7645 /work/haukurpj/data/formatted/Parice1.0/test.en
7645 /work/haukurpj/data/formatted/Parice1.0/test.is


In [27]:
!rm -rf {parice_formatted_dir}/prófunarsett

In [28]:
!ls {target_dir}

baekur.en   dev.is  eso.en	  hagstofan.is	    tatoeba.en	train.is
baekur.is   ees.en  eso.is	  kde4.en	    tatoeba.is	ubuntu.en
biblian.en  ees.is  fornritin.en  kde4.is	    test.en	ubuntu.is
biblian.is  ema.en  fornritin.is  opensubtitles.en  test.is
dev.en	    ema.is  hagstofan.en  opensubtitles.is  train.en
